In [20]:
from __future__ import absolute_import
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.layers import Conv3D
import tensorflow as tf

class GroupDepthwiseConv3D(Conv3D):
    def __init__(self,
                 kernel_size,
                 strides=(1, 1, 1),
                 padding='valid',
                 group_multiplier=1,
                 group_size=1,
                 data_format=None,
                 activation=None,
                 use_bias=True,
                 group_depthwise_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 dilation_rate=(1, 1, 1),
                 group_depthwise_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 group_depthwise_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(GroupDepthwiseConv3D, self).__init__(
            filters=None,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            bias_regularizer=bias_regularizer,
            dilation_rate=dilation_rate,
            activity_regularizer=activity_regularizer,
            bias_constraint=bias_constraint,
            **kwargs)

        self.group_multiplier = group_multiplier
        self.group_size = group_size
        self.group_depthwise_initializer = initializers.get(group_depthwise_initializer)
        self.group_depthwise_regularizer = regularizers.get(group_depthwise_regularizer)
        self.group_depthwise_constraint = constraints.get(group_depthwise_constraint)
        self.bias_initializer = initializers.get(bias_initializer)
        self.dilation_rate = dilation_rate
        self._padding = "VALID"
        self._strides = (1,) + self.strides + (1,)
        if self.data_format == 'channels_first':
            self.channel_axis = 1
            self._data_format = "NCDHW"
        else:
            self.channel_axis = -1
            self._data_format = "NDHWC"
        self.input_dim = None
        self.kernel = None
        self.bias = None
        self.group_num = None

    def build(self, input_shape):

        self.input_dim = int(input_shape[self.channel_axis])

        if self.input_dim % self.group_size != 0:
            raise ValueError('input channel num should be'
                             'integral multiple of group_size.')

        self.group_num = self.input_dim // self.group_size

        kernel_shape = (self.group_num,
                        self.kernel_size[0],
                        self.kernel_size[1],
                        self.kernel_size[2],
                        self.group_size,
                        self.group_multiplier
                       )

        self.kernel = self.add_weight(
            shape=kernel_shape,
            initializer=self.group_depthwise_initializer,
            name='kernel',
            regularizer=self.group_depthwise_regularizer,
            constraint=self.group_depthwise_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.group_num * self.group_multiplier,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
    def call(self, inputs, training=True):
        layers = []
        for i in range(self.group_num):
            exp = tf.expand_dims(inputs[:, :, :, :, i], axis = -1)
            conv_l = tf.keras.layers.Conv3D(1, kernel_size = self.kernel_size, strides = self.strides, padding = self.padding)(exp)
            layers.append(conv_l)
        concat = tf.keras.layers.Concatenate(axis = -1)(layers)
        conv = tf.keras.layers.Conv3D(inputs.shape[-1], (1, 1, 1))(concat)
        return conv

In [21]:
input_layer = tf.keras.layers.Input((64, 57, 57, 1024))
depth = GroupDepthwiseConv3D(kernel_size = (1, 3, 3), strides = (2, 2, 2), padding = "valid")(input_layer)
print(depth.shape)

(None, 32, 28, 28, 1024)


In [22]:
import tensorflow as tf

In [23]:
# def GroupDepthwiseConv3D(input_layer, kernel_size, strides):
#     layers = []
#     num = input_layer.shape[-1]
#     for i in range(num):
#         exp = tf.expand_dims(input_layer[:, :, :, :, i], axis = -1)
#         conv_l = tf.keras.layers.Conv3D(1, kernel_size = kernel_size, strides = strides)(exp)
#         layers.append(conv_l)
#     concat = tf.keras.layers.Concatenate(axis = -1)(layers)
# #     conv = tf.keras.layers.Conv3D(input_layer.shape[-1], (1, 1, 1))(concat)
#     return concat

In [24]:
# def GroupDepthwiseConv3D(input_layer, kernel_size, strides):
#     layers = []
#     num = input_layer.shape[1]
#     for i in range(num):
#         conv_l = tf.keras.layers.DepthwiseConv2D(kernel_size, strides)(input_layer[:, i, :, :, :])
#         conv_l = tf.expand_dims(conv_l, axis = 1)
#         layers.append(conv_l)
#     concat = tf.keras.layers.Concatenate(axis = 1)(layers)

#     return concat

In [25]:
# input_layer = tf.keras.layers.Input((64, 224, 224, 1024))
# anunay = GroupDepthwiseConv3D(input_layer, (3, 3, 3), (1, 1, 1), 8)
# print(anunay.shape)

In [26]:
def module_1(input_layer, kernel_size, strides, activation, padding = "same"):
    depth = GroupDepthwiseConv3D(kernel_size, strides, padding = padding)(input_layer)
    bn = tf.keras.layers.BatchNormalization()(depth)
    activation = tf.keras.layers.Activation(activation)(bn)
    return activation

def module_2(input_layer, kernel_size_1, strides_1, kernel_size_2, strides_2, activation_1, activation_2, padding):
    depth_1 = GroupDepthwiseConv3D(input_layer, kernel_size_1, strides_1)
    
    bn_1 = tf.keras.layers.BatchNormalization()(depth_1)
    activation_1 = tf.keras.layers.Activation(activation_1)(bn_1)
    zero_padding = tf.keras.layers.ZeroPadding3D(padding)(activation_1)
    
    depth_2 = GroupDepthwiseConv3D(zero_padding, kernel_size_2, strides_2)
    
    bn_2 = tf.keras.layers.BatchNormalization()(depth_2)
    activation_2 = tf.keras.layers.Activation(activation_2)(bn_2)
    return activation_2

def module_3(input_layer, filters_1, kernel_size_1, strides_1, filters_2, kernel_size_2, strides_2):
    gap = tf.keras.layers.GlobalAveragePooling3D()(input_layer)
    rescale = tf.keras.layers.Reshape((1, 1, 1, input_layer.shape[-1]))(gap)
    conv_1 = tf.keras.layers.Conv3D(filters_1, kernel_size_1, strides=strides_1, padding='valid', groups=1, activation=None)(rescale)
    conv_2 = tf.keras.layers.Conv3D(filters_2, kernel_size_2, strides=strides_2, padding='valid', groups=1, activation=None)(conv_1)
    return conv_2

def stem(input_layer, padding, filters, kernel_size, strides, activations):
    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
    norm = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(rescale)
    zero = tf.keras.layers.ZeroPadding3D(padding)(norm)
    conv = tf.keras.layers.Conv3D(filters, kernel_size, strides)(zero)
    bn = tf.keras.layers.BatchNormalization()(conv)
    activation = tf.keras.layers.Activation(activations)(bn)
    return activation

def final(input_layer, filters, kernel_size, strides, padding, activations):
    conv = tf.keras.layers.Conv3D(filters, kernel_size, strides, padding)(input_layer)
    bn = tf.keras.layers.BatchNormalization()(conv)
    activation = tf.keras.layers.Activation(activations)(bn)
    return activation

EfficientNet - B0

In [45]:
def EfficientNet0_3D(input_layer):
    start = stem(input_layer, (0, 1, 1), 32, (1, 3, 3), (2, 2, 2), "relu")
    ### 1a ###
    mod_1 = module_1(start, (3, 3, 3), (1, 1, 1), "relu", "same")###
    mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 32, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2])
    conv_1a = tf.keras.layers.Conv3D(16, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
    ### 2a ###
    conv_2a = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_1a)
    bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
    activation_2a = tf.keras.layers.Activation("relu")(bn_2a)
    zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
    mod_1 = module_1(zero_2a, (1, 3, 3), (1, 2, 2), "relu", "valid")
#     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 4, (1, 1, 1), (1, 1, 1), 96, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_2a2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
    ### 2b ###
    conv_2b = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
    bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
    activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
    mod_1 = module_1(activation_2b, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_2b2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
    drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
    add = tf.keras.layers.Add()([bn_2a2, drop_2b])
    
    ### 3a ###
    conv_3a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
    activation_3a = tf.keras.layers.Activation("relu")(bn_3a)
    zero_3a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_3a)
    mod_1 = module_1(zero_3a, (1, 3, 3), (2, 2, 2), "relu", "valid")
#     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_3a2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
    ### 3b ###
    conv_3b = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
    bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
    activation_3b = tf.keras.layers.Activation("relu")(bn_3b)
    mod_1 = module_1(activation_3b, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_3b2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
    drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
    add = tf.keras.layers.Add()([bn_3a2, drop_3b])
    ### 4a ###
    conv_4a = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
    activation_4a = tf.keras.layers.Activation("relu")(bn_4a)
    zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
    mod_1 = module_1(zero_4a, (1, 3, 3), (2, 2, 2), "relu", "valid")
#     mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4a2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
    ### 4b ###
    conv_4b = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
    bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
    activation_4b = tf.keras.layers.Activation("relu")(bn_4b)
    mod_1 = module_1(activation_4b, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4b2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
    drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
    add = tf.keras.layers.Add()([bn_4a2, drop_4b])
    ### 4c ###
    conv_4c = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
    activation_4c = tf.keras.layers.Activation("relu")(bn_4c)
    mod_1 = module_1(activation_4c, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4c2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
    drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
    add = tf.keras.layers.Add()([add, drop_4c])
    ### 5a ###
    conv_5a = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
    activation_5a = tf.keras.layers.Activation("relu")(bn_5a)
    mod_1 = module_1(activation_5a, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5a2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
    ### 5b ###
    conv_5b = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
    bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
    activation_5b = tf.keras.layers.Activation("relu")(bn_5b)
    mod_1 = module_1(activation_5b, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5b2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
    drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
    add = tf.keras.layers.Add()([bn_5a2, drop_5b])
    ### 5c ###
    conv_5c = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
    activation_5c = tf.keras.layers.Activation("relu")(bn_5c)
    mod_1 = module_1(activation_5c, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5c2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
    drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
    add = tf.keras.layers.Add()([add, drop_5c])
    ### 6a ###
    conv_6a = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
    activation_6a = tf.keras.layers.Activation("relu")(bn_6a)
    zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
    mod_1 = module_1(zero_6a, (1, 3, 3), (2, 2, 2), "relu", "same")
#     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6a2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
    ### 6b ###
    conv_6b = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
    bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
    activation_6b = tf.keras.layers.Activation("relu")(bn_6b)
    mod_1 = module_1(activation_6b, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6b2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
    drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
    add = tf.keras.layers.Add()([bn_6a2, drop_6b])
    ## 6c ###
    conv_6c = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
    activation_6c = tf.keras.layers.Activation("relu")(bn_6c)
    mod_1 = module_1(activation_6c, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6c2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
    drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
    add = tf.keras.layers.Add()([add, drop_6c])
    ### 6d ###
    conv_6d = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
    activation_6d = tf.keras.layers.Activation("relu")(bn_6d)
    mod_1 = module_1(activation_6d, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6d2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
    drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
    add = tf.keras.layers.Add()([add, drop_6d])
    ### 7a ###
    conv_7a = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
    activation_7a = tf.keras.layers.Activation("relu")(bn_7a)
    mod_1 = module_1(activation_7a, (3, 3, 3), (1, 1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_7a2 = tf.keras.layers.Conv3D(320, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
  
    end = final(add, 1280, (1, 1, 1), (1, 1, 1), "valid", "relu")
    
    return end

In [46]:
input_layer = tf.keras.layers.Input((64, 224, 224, 3))
output = EfficientNet0_3D(input_layer)
print(output.shape)

(None, 4, 8, 8, 1280)


In [47]:
model = tf.keras.models.Model(inputs = input_layer, outputs = output)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 64, 224, 224 0                                            
__________________________________________________________________________________________________
rescaling_8 (Rescaling)         (None, 64, 224, 224, 0           input_17[0][0]                   
__________________________________________________________________________________________________
normalization_8 (Normalization) (None, 64, 224, 224, 7           rescaling_8[0][0]                
__________________________________________________________________________________________________
zero_padding3d_39 (ZeroPadding3 (None, 64, 226, 226, 0           normalization_8[0][0]            
____________________________________________________________________________________________

In [39]:
eff = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, input_shape=(224, 224, 3))

16719872/16705208 [==============================] - 5s 0us/step


In [40]:
eff.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_5 (Rescaling)         (None, 224, 224, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
normalization_5 (Normalization) (None, 224, 224, 3)  7           rescaling_5[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_5[0][0]            
_____________________________________________________________________________________

In [90]:
def EfficientNet1_3D(input_layer):
    start = stem(input_layer, (0, 1, 1), 32, (1, 3, 3), (2, 2, 2), "relu")
    ### 1a ###
    mod_1 = module_1(start, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 32, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2])
    conv_1a = tf.keras.layers.Conv3D(16, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
    ### Block_1b ###
    mod_1 = module_1(bn_1a, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 4, (1, 1, 1), (1, 1, 1), 16, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_1b = tf.keras.layers.Conv3D(16, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_1b = tf.keras.layers.BatchNormalization()(conv_1b)
    drop_1b = tf.keras.layers.Dropout(0.2)(bn_1b)
    add = tf.keras.layers.Add()([bn_1a, drop_1b])
    ### 2a ###
    conv_2a = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
    activation_2a = tf.keras.layers.Activation("relu")(bn_2a)
    zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
    mod_1 = module_1(zero_2a, (3, 3), (2, 2), "relu")
    mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 4, (1, 1, 1), (1, 1, 1), 96, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_2a2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
    ### 2b ###
    conv_2b = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
    bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
    activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
    mod_1 = module_1(activation_2b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_2b2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
    drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
    add = tf.keras.layers.Add()([bn_2a2, drop_2b])
    ### Block_2c ###
    conv_2c = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_2c = tf.keras.layers.BatchNormalization()(conv_2c)
    activation_2c = tf.keras.layers.Activation("relu")(bn_2c)
    mod_1 = module_1(activation_2c, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_2c2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_2c2 = tf.keras.layers.BatchNormalization()(conv_2c2)
    drop_2c = tf.keras.layers.Dropout(0.2)(bn_2c2)
    add = tf.keras.layers.Add()([add, drop_2c])
    ### 3a ###
    conv_3a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
    activation_3a = tf.keras.layers.Activation("relu")(bn_3a)
    zero_3a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_3a)
    mod_1 = module_1(zero_3a, (3, 3), (2, 2), "relu")
    mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_3a2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
    ### 3b ###
    conv_3b = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
    bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
    activation_3b = tf.keras.layers.Activation("relu")(bn_3b)
    mod_1 = module_1(activation_3b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_3b2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
    drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
    add = tf.keras.layers.Add()([bn_3a2, drop_3b])
    ### Block_3c ###
    conv_3c = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_3c = tf.keras.layers.BatchNormalization()(conv_3c)
    activation_3c = tf.keras.layers.Activation("relu")(bn_3c)
    mod_1 = module_1(activation_3c, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_3c2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_3c2 = tf.keras.layers.BatchNormalization()(conv_3c2)
    drop_3c = tf.keras.layers.Dropout(0.2)(bn_3c2)
    add = tf.keras.layers.Add()([add, drop_3c])
    ### 4a ###
    conv_4a = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
    activation_4a = tf.keras.layers.Activation("relu")(bn_4a)
    zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
    mod_1 = module_1(zero_4a, (3, 3), (2, 2), "relu")
    mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4a2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
    ### 4b ###
    conv_4b = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
    bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
    activation_4b = tf.keras.layers.Activation("relu")(bn_4b)
    mod_1 = module_1(activation_4b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4b2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
    drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
    add = tf.keras.layers.Add()([bn_4a2, drop_4b])
    ### 4c ###
    conv_4c = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
    activation_4c = tf.keras.layers.Activation("relu")(bn_4c)
    mod_1 = module_1(activation_4c, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4c2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
    drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
    add = tf.keras.layers.Add()([add, drop_4c])
    ### Block_4d ###
    conv_4d = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_4d = tf.keras.layers.BatchNormalization()(conv_4d)
    activation_4d = tf.keras.layers.Activation("relu")(bn_4d)
    mod_1 = module_1(activation_4d, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_4d2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_4d2 = tf.keras.layers.BatchNormalization()(conv_4d2)
    drop_4d = tf.keras.layers.Dropout(0.2)(bn_4d2)
    add = tf.keras.layers.Add()([add, drop_4d])
  
    ### 5a ###
    conv_5a = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
    activation_5a = tf.keras.layers.Activation("relu")(bn_5a)
    mod_1 = module_1(activation_5a, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5a2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
    ### 5b ###
    conv_5b = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
    bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
    activation_5b = tf.keras.layers.Activation("relu")(bn_5b)
    mod_1 = module_1(activation_5b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5b2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
    drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
    add = tf.keras.layers.Add()([bn_5a2, drop_5b])
    ### 5c ###
    conv_5c = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
    activation_5c = tf.keras.layers.Activation("relu")(bn_5c)
    mod_1 = module_1(activation_5c, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5c2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
    drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
    add = tf.keras.layers.Add()([add, drop_5c])
    ### Block_5d ###
    conv_5d = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_5d = tf.keras.layers.BatchNormalization()(conv_5d)
    activation_5d = tf.keras.layers.Activation("relu")(bn_5d)
    mod_1 = module_1(activation_5d, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_5d2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_5d2 = tf.keras.layers.BatchNormalization()(conv_5d2)
    drop_5d = tf.keras.layers.Dropout(0.2)(bn_5d2)
    add = tf.keras.layers.Add()([add, drop_5d])
  
    ### 6a ###
    conv_6a = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
    activation_6a = tf.keras.layers.Activation("relu")(bn_6a)
    zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
    mod_1 = module_1(zero_6a, (3, 3), (2, 2), "relu")
    mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
    mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6a2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
    ### 6b ###
    conv_6b = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
    bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
    activation_6b = tf.keras.layers.Activation("relu")(bn_6b)
    mod_1 = module_1(activation_6b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6b2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
    drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
    add = tf.keras.layers.Add()([bn_6a2, drop_6b])
    ## 6c ###
    conv_6c = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
    activation_6c = tf.keras.layers.Activation("relu")(bn_6c)
    mod_1 = module_1(activation_6c, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6c2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
    drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
    add = tf.keras.layers.Add()([add, drop_6c])
    ### 6d ###
    conv_6d = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
    activation_6d = tf.keras.layers.Activation("relu")(bn_6d)
    mod_1 = module_1(activation_6d, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6d2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
    drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
    add = tf.keras.layers.Add()([add, drop_6d])
    ### Block_6e ###
    conv_6e = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_6e = tf.keras.layers.BatchNormalization()(conv_6e)
    activation_6e = tf.keras.layers.Activation("relu")(bn_6e)
    mod_1 = module_1(activation_6e, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_6e2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_6e2 = tf.keras.layers.BatchNormalization()(conv_6e2)
    drop_6e = tf.keras.layers.Dropout(0.2)(bn_6e2)
    add = tf.keras.layers.Add()([add, drop_6e])
  
    ### 7a ###
    conv_7a = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
    bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
    activation_7a = tf.keras.layers.Activation("relu")(bn_7a)
    mod_1 = module_1(activation_7a, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_7a2 = tf.keras.layers.Conv3D(320, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
    ### Block_7b ###
    conv_7b = tf.keras.layers.Conv3D(1920, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_7a2)
    bn_7b = tf.keras.layers.BatchNormalization()(conv_7b)
    activation_7b = tf.keras.layers.Activation("relu")(bn_7b)
    mod_1 = module_1(activation_7b, (1, 1), (1, 1), "relu")
    mod_2 = module_3(mod_1, 80, (1, 1, 1), (1, 1, 1), 1920, (1, 1, 1), (1, 1, 1))
    mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
    conv_7b2 = tf.keras.layers.Conv3D(320, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
    bn_7b2 = tf.keras.layers.BatchNormalization()(conv_7b2)
    drop_7b = tf.keras.layers.Dropout(0.2)(bn_7b2)
    add = tf.keras.layers.Add()([bn_7a2, drop_7b])

    end = final(add, 1280, (1, 1, 1), (1, 1, 1), "valid", "relu")
    
    return end

In [91]:
input_layer = tf.keras.layers.Input((64, 224, 224, 3))
anunay = EfficientNet0_3D(input_layer)
print(anunay.shape)

(None, 4, 7, 7, 1280)


EFFICIENT NET - 3(3D)

In [48]:
def EfficientNet3_3D(input_layer):
  start = stem(input_layer, (0, 1, 1), 40, (3, 3, 3), (2, 2, 2), "relu")
  ### Block_1a ###
  mod_1 = module_1(start, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 40, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2])
  conv_1a = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
  ### Block_1b ###
  mod_1 = module_1(bn_1a, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 24, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_1b = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1b = tf.keras.layers.BatchNormalization()(conv_1b)
  drop_1b = tf.keras.layers.Dropout(0.2)(bn_1b)
  add = tf.keras.layers.Add()([bn_1a, drop_1b])
  ### Block_2a ###
  conv_2a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
  activation_2a = tf.keras.layers.Activation("relu")(bn_2a)
  zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
  mod_1 = module_1(zero_2a, (3, 3), (2, 2), "relu")#################
  mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2a2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
  ### Block_2b ###
  conv_2b = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
  bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
  activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
  mod_1 = module_1(activation_2b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2b2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
  drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
  add = tf.keras.layers.Add()([bn_2a2, drop_2b])
  ### Block_2c ###
  conv_2c = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2c = tf.keras.layers.BatchNormalization()(conv_2c)
  activation_2c = tf.keras.layers.Activation("relu")(bn_2c)
  mod_1 = module_1(activation_2c, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2c2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2c2 = tf.keras.layers.BatchNormalization()(conv_2c2)
  drop_2c = tf.keras.layers.Dropout(0.2)(bn_2c2)
  add = tf.keras.layers.Add()([add, drop_2c])

  ### Block_3a ###
  conv_3a = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
  activation_3a = tf.keras.layers.Activation("relu")(bn_3a)
  zero_3a = tf.keras.layers.ZeroPadding3D((0, 2, 2))(activation_3a)
  mod_1 = module_1(zero_3a, (4, 4), (2, 2), "relu")##############################################3
  mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3a2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
  ### Block_3b ###
  conv_3b = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
  bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
  activation_3b = tf.keras.layers.Activation("relu")(bn_3b)
  mod_1 = module_1(activation_3b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3b2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
  drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
  add = tf.keras.layers.Add()([bn_3a2, drop_3b])
  ### Block_3c ###
  conv_3c = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3c = tf.keras.layers.BatchNormalization()(conv_3c)
  activation_3c = tf.keras.layers.Activation("relu")(bn_3c)
  mod_1 = module_1(activation_3c, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3c2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3c2 = tf.keras.layers.BatchNormalization()(conv_3c2)
  drop_3c = tf.keras.layers.Dropout(0.2)(bn_3c2)
  add = tf.keras.layers.Add()([add, drop_3c])

  ### Block_4a ###
  conv_4a = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
  activation_4a = tf.keras.layers.Activation("relu")(bn_4a)
  zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
  mod_1 = module_1(zero_4a, (3, 3), (2, 2), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4a2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
  ### Block_4b ###
  conv_4b = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
  bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
  activation_4b = tf.keras.layers.Activation("relu")(bn_4b)
  mod_1 = module_1(activation_4b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4b2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
  drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
  add = tf.keras.layers.Add()([bn_4a2, drop_4b])
  ### Block_4c ###
  conv_4c = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
  activation_4c = tf.keras.layers.Activation("relu")(bn_4c)
  mod_1 = module_1(activation_4c, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4c2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
  drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
  add = tf.keras.layers.Add()([add, drop_4c])
  ### Block_4d ###
  conv_4d = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4d = tf.keras.layers.BatchNormalization()(conv_4d)
  activation_4d = tf.keras.layers.Activation("relu")(bn_4d)
  mod_1 = module_1(activation_4d, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4d2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4d2 = tf.keras.layers.BatchNormalization()(conv_4d2)
  drop_4d = tf.keras.layers.Dropout(0.2)(bn_4d2)
  add = tf.keras.layers.Add()([add, drop_4d])
  ### Block_4e ###
  conv_4e = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4e = tf.keras.layers.BatchNormalization()(conv_4e)
  activation_4e = tf.keras.layers.Activation("relu")(bn_4e)
  mod_1 = module_1(activation_4e, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4e2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4e2 = tf.keras.layers.BatchNormalization()(conv_4e2)
  drop_4e = tf.keras.layers.Dropout(0.2)(bn_4e2)
  add = tf.keras.layers.Add()([add, drop_4e])

  ### Block_5a ###
  conv_5a = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
  activation_5a = tf.keras.layers.Activation("relu")(bn_5a)
  mod_1 = module_1(activation_5a, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5a2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
  ### Block_5b ###
  conv_5b = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
  bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
  activation_5b = tf.keras.layers.Activation("relu")(bn_5b)
  mod_1 = module_1(activation_5b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5b2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
  drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
  add = tf.keras.layers.Add()([bn_5a2, drop_5b])
  ### Block_5c ###
  conv_5c = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
  activation_5c = tf.keras.layers.Activation("relu")(bn_5c)
  mod_1 = module_1(activation_5c, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5c2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
  drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
  add = tf.keras.layers.Add()([add, drop_5c])
  ### Block_5d ###
  conv_5d = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5d = tf.keras.layers.BatchNormalization()(conv_5d)
  activation_5d = tf.keras.layers.Activation("relu")(bn_5d)
  mod_1 = module_1(activation_5d, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5d2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5d2 = tf.keras.layers.BatchNormalization()(conv_5d2)
  drop_5d = tf.keras.layers.Dropout(0.2)(bn_5d2)
  add = tf.keras.layers.Add()([add, drop_5d])
  ### Block_5e ###
  conv_5e = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5e = tf.keras.layers.BatchNormalization()(conv_5e)
  activation_5e = tf.keras.layers.Activation("relu")(bn_5e)
  mod_1 = module_1(activation_5e, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5e2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5e2 = tf.keras.layers.BatchNormalization()(conv_5e2)
  drop_5e = tf.keras.layers.Dropout(0.2)(bn_5e2)
  add = tf.keras.layers.Add()([add, drop_5e])

  ### Block_6a ###
  conv_6a = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
  activation_6a = tf.keras.layers.Activation("relu")(bn_6a)
  zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
  mod_1 = module_1(zero_6a, (3, 3), (2, 2), "relu")
  mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6a2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
  ### Block_6b ###
  conv_6b = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
  bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
  activation_6b = tf.keras.layers.Activation("relu")(bn_6b)
  mod_1 = module_1(activation_6b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6b2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
  drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
  add = tf.keras.layers.Add()([bn_6a2, drop_6b])
  ### Block_6c ###
  conv_6c = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
  activation_6c = tf.keras.layers.Activation("relu")(bn_6c)
  mod_1 = module_1(activation_6c, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6c2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
  drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
  add = tf.keras.layers.Add()([add, drop_6c])
  ### Block_6d ###
  conv_6d = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
  activation_6d = tf.keras.layers.Activation("relu")(bn_6d)
  mod_1 = module_1(activation_6d, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6d2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
  drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
  add = tf.keras.layers.Add()([add, drop_6d])
  ### Block_6e ###
  conv_6e = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6e = tf.keras.layers.BatchNormalization()(conv_6e)
  activation_6e = tf.keras.layers.Activation("relu")(bn_6e)
  mod_1 = module_1(activation_6e, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6e2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6e2 = tf.keras.layers.BatchNormalization()(conv_6e2)
  drop_6e = tf.keras.layers.Dropout(0.2)(bn_6e2)
  add = tf.keras.layers.Add()([add, drop_6e])
  ### Block_6f ###
  conv_6f = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6f = tf.keras.layers.BatchNormalization()(conv_6f)
  activation_6f = tf.keras.layers.Activation("relu")(bn_6f)
  mod_1 = module_1(activation_6f, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6f2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6f2 = tf.keras.layers.BatchNormalization()(conv_6f2)
  drop_6f = tf.keras.layers.Dropout(0.2)(bn_6f2)
  add = tf.keras.layers.Add()([add, drop_6f])

  ### Block_7a ###
  conv_7a = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
  activation_7a = tf.keras.layers.Activation("relu")(bn_7a)
  mod_1 = module_1(activation_7a, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7a2 = tf.keras.layers.Conv3D(384, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
  ### Block_7b ###
  conv_7b = tf.keras.layers.Conv3D(2304, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_7a2)
  bn_7b = tf.keras.layers.BatchNormalization()(conv_7b)
  activation_7b = tf.keras.layers.Activation("relu")(bn_7b)
  mod_1 = module_1(activation_7b, (1, 1), (1, 1), "relu")
  mod_2 = module_3(mod_1, 96, (1, 1, 1), (1, 1, 1), 2304, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7b2 = tf.keras.layers.Conv3D(384, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7b2 = tf.keras.layers.BatchNormalization()(conv_7b2)
  drop_7b = tf.keras.layers.Dropout(0.2)(bn_7b2)
  add = tf.keras.layers.Add()([bn_7a2, drop_7b])

  ### END ###
  end = final(add, 1536, (1, 1, 1), (1, 1, 1), "valid", "relu")
  # gap = tf.keras.layers.GlobalAveragePooling
  return end

In [57]:
def EfficientNet3_3D(input_layer):
  start = stem(input_layer, (0, 1, 1), 40, (3, 3, 3), (2, 2, 2), "relu")
  ### Block_1a ###
  mod_1 = module_1(start, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 40, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2])
  conv_1a = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
  ### Block_1b ###
  mod_1 = module_1(bn_1a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 24, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_1b = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1b = tf.keras.layers.BatchNormalization()(conv_1b)
  drop_1b = tf.keras.layers.Dropout(0.2)(bn_1b)
  add = tf.keras.layers.Add()([bn_1a, drop_1b])
  ### Block_2a ###
  conv_2a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
  activation_2a = tf.keras.layers.Activation("relu")(bn_2a)
  zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
  mod_1 = module_1(zero_2a, (1, 3, 3), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2a2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
  ### Block_2b ###
  conv_2b = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
  bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
  activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
  mod_1 = module_1(activation_2b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2b2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
  drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
  add = tf.keras.layers.Add()([bn_2a2, drop_2b])
  ### Block_2c ###
  conv_2c = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2c = tf.keras.layers.BatchNormalization()(conv_2c)
  activation_2c = tf.keras.layers.Activation("relu")(bn_2c)
  mod_1 = module_1(activation_2c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2c2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2c2 = tf.keras.layers.BatchNormalization()(conv_2c2)
  drop_2c = tf.keras.layers.Dropout(0.2)(bn_2c2)
  add = tf.keras.layers.Add()([add, drop_2c])

  ### Block_3a ###
  conv_3a = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
  activation_3a = tf.keras.layers.Activation("relu")(bn_3a)
  zero_3a = tf.keras.layers.ZeroPadding3D((0, 2, 2))(activation_3a)
  mod_1 = module_1(zero_3a, (1, 4, 4), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3a2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
  ### Block_3b ###
  conv_3b = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
  bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
  activation_3b = tf.keras.layers.Activation("relu")(bn_3b)
  mod_1 = module_1(activation_3b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3b2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
  drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
  add = tf.keras.layers.Add()([bn_3a2, drop_3b])
  ### Block_3c ###
  conv_3c = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3c = tf.keras.layers.BatchNormalization()(conv_3c)
  activation_3c = tf.keras.layers.Activation("relu")(bn_3c)
  mod_1 = module_1(activation_3c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3c2 = tf.keras.layers.Conv3D(48, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3c2 = tf.keras.layers.BatchNormalization()(conv_3c2)
  drop_3c = tf.keras.layers.Dropout(0.2)(bn_3c2)
  add = tf.keras.layers.Add()([add, drop_3c])

  ### Block_4a ###
  conv_4a = tf.keras.layers.Conv3D(288, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
  activation_4a = tf.keras.layers.Activation("relu")(bn_4a)
  zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
  mod_1 = module_1(zero_4a, (1, 3, 3), (1, 2, 2), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 288, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4a2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
  ### Block_4b ###
  conv_4b = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
  bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
  activation_4b = tf.keras.layers.Activation("relu")(bn_4b)
  mod_1 = module_1(activation_4b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4b2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
  drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
  add = tf.keras.layers.Add()([bn_4a2, drop_4b])
  ### Block_4c ###
  conv_4c = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
  activation_4c = tf.keras.layers.Activation("relu")(bn_4c)
  mod_1 = module_1(activation_4c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4c2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
  drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
  add = tf.keras.layers.Add()([add, drop_4c])
  ### Block_4d ###
  conv_4d = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4d = tf.keras.layers.BatchNormalization()(conv_4d)
  activation_4d = tf.keras.layers.Activation("relu")(bn_4d)
  mod_1 = module_1(activation_4d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4d2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4d2 = tf.keras.layers.BatchNormalization()(conv_4d2)
  drop_4d = tf.keras.layers.Dropout(0.2)(bn_4d2)
  add = tf.keras.layers.Add()([add, drop_4d])
  ### Block_4e ###
  conv_4e = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4e = tf.keras.layers.BatchNormalization()(conv_4e)
  activation_4e = tf.keras.layers.Activation("relu")(bn_4e)
  mod_1 = module_1(activation_4e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4e2 = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4e2 = tf.keras.layers.BatchNormalization()(conv_4e2)
  drop_4e = tf.keras.layers.Dropout(0.2)(bn_4e2)
  add = tf.keras.layers.Add()([add, drop_4e])

  ### Block_5a ###
  conv_5a = tf.keras.layers.Conv3D(576, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
  activation_5a = tf.keras.layers.Activation("relu")(bn_5a)
  mod_1 = module_1(activation_5a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 24, (1, 1, 1), (1, 1, 1), 576, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5a2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
  ### Block_5b ###
  conv_5b = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
  bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
  activation_5b = tf.keras.layers.Activation("relu")(bn_5b)
  mod_1 = module_1(activation_5b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5b2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
  drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
  add = tf.keras.layers.Add()([bn_5a2, drop_5b])
  ### Block_5c ###
  conv_5c = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
  activation_5c = tf.keras.layers.Activation("relu")(bn_5c)
  mod_1 = module_1(activation_5c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5c2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
  drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
  add = tf.keras.layers.Add()([add, drop_5c])
  ### Block_5d ###
  conv_5d = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5d = tf.keras.layers.BatchNormalization()(conv_5d)
  activation_5d = tf.keras.layers.Activation("relu")(bn_5d)
  mod_1 = module_1(activation_5d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5d2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5d2 = tf.keras.layers.BatchNormalization()(conv_5d2)
  drop_5d = tf.keras.layers.Dropout(0.2)(bn_5d2)
  add = tf.keras.layers.Add()([add, drop_5d])
  ### Block_5e ###
  conv_5e = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5e = tf.keras.layers.BatchNormalization()(conv_5e)
  activation_5e = tf.keras.layers.Activation("relu")(bn_5e)
  mod_1 = module_1(activation_5e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5e2 = tf.keras.layers.Conv3D(136, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5e2 = tf.keras.layers.BatchNormalization()(conv_5e2)
  drop_5e = tf.keras.layers.Dropout(0.2)(bn_5e2)
  add = tf.keras.layers.Add()([add, drop_5e])

  ### Block_6a ###
  conv_6a = tf.keras.layers.Conv3D(816, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
  activation_6a = tf.keras.layers.Activation("relu")(bn_6a)
  zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
  mod_1 = module_1(zero_6a, (1, 3, 3), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 34, (1, 1, 1), (1, 1, 1), 816, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6a2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
  ### Block_6b ###
  conv_6b = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
  bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
  activation_6b = tf.keras.layers.Activation("relu")(bn_6b)
  mod_1 = module_1(activation_6b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6b2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
  drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
  add = tf.keras.layers.Add()([bn_6a2, drop_6b])
  ### Block_6c ###
  conv_6c = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
  activation_6c = tf.keras.layers.Activation("relu")(bn_6c)
  mod_1 = module_1(activation_6c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6c2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
  drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
  add = tf.keras.layers.Add()([add, drop_6c])
  ### Block_6d ###
  conv_6d = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
  activation_6d = tf.keras.layers.Activation("relu")(bn_6d)
  mod_1 = module_1(activation_6d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6d2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
  drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
  add = tf.keras.layers.Add()([add, drop_6d])
  ### Block_6e ###
  conv_6e = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6e = tf.keras.layers.BatchNormalization()(conv_6e)
  activation_6e = tf.keras.layers.Activation("relu")(bn_6e)
  mod_1 = module_1(activation_6e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6e2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6e2 = tf.keras.layers.BatchNormalization()(conv_6e2)
  drop_6e = tf.keras.layers.Dropout(0.2)(bn_6e2)
  add = tf.keras.layers.Add()([add, drop_6e])
  ### Block_6f ###
  conv_6f = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6f = tf.keras.layers.BatchNormalization()(conv_6f)
  activation_6f = tf.keras.layers.Activation("relu")(bn_6f)
  mod_1 = module_1(activation_6f, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6f2 = tf.keras.layers.Conv3D(232, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6f2 = tf.keras.layers.BatchNormalization()(conv_6f2)
  drop_6f = tf.keras.layers.Dropout(0.2)(bn_6f2)
  add = tf.keras.layers.Add()([add, drop_6f])

  ### Block_7a ###
  conv_7a = tf.keras.layers.Conv3D(1392, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
  activation_7a = tf.keras.layers.Activation("relu")(bn_7a)
  mod_1 = module_1(activation_7a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 58, (1, 1, 1), (1, 1, 1), 1392, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7a2 = tf.keras.layers.Conv3D(384, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
  ### Block_7b ###
  conv_7b = tf.keras.layers.Conv3D(2304, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_7a2)
  bn_7b = tf.keras.layers.BatchNormalization()(conv_7b)
  activation_7b = tf.keras.layers.Activation("relu")(bn_7b)
  mod_1 = module_1(activation_7b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 96, (1, 1, 1), (1, 1, 1), 2304, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7b2 = tf.keras.layers.Conv3D(384, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7b2 = tf.keras.layers.BatchNormalization()(conv_7b2)
  drop_7b = tf.keras.layers.Dropout(0.2)(bn_7b2)
  add = tf.keras.layers.Add()([bn_7a2, drop_7b])

  ### END ###
  end = final(add, 1536, (1, 1, 1), (1, 1, 1), "valid", "relu")
  # gap = tf.keras.layers.GlobalAveragePooling
  return end

In [58]:
input_layer = tf.keras.layers.Input((64, 224, 224, 3))
anunay = EfficientNet3_3D(input_layer)
print(anunay.shape)

KeyboardInterrupt: 

In [ ]:
model_3 = tf.keras.models.Model(inputs = input_layer, outputs = anunay)
model_3.summary()

EFFICIENT NET - 4(3D)

In [8]:
def EfficientNet4_3D(input_layer):
  start = stem(input_layer, (0, 1, 1), 48, (3, 3, 3), (2, 2, 2), "relu")
  ### Block_1a ###
  mod_1 = module_1(start, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 12, (1, 1, 1), (1, 1, 1), 48, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2])
  conv_1a = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
  ### Block_1b ###
  mod_1 = module_1(bn_1a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 24, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_1b = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_1b = tf.keras.layers.BatchNormalization()(conv_1b)
  drop_1b = tf.keras.layers.Dropout(0.2)(bn_1b)
  add = tf.keras.layers.Add()([bn_1a, drop_1b])
  ### Block_2a ###
  conv_2a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
  activation_2a = tf.keras.layers.Activation("relu")(bn_2a)
  zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
  mod_1 = module_1(zero_2a, (1, 3, 3), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2a2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
  ### Block_2b ###
  conv_2b = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
  bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
  activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
  mod_1 = module_1(activation_2b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2b2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
  drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
  add = tf.keras.layers.Add()([bn_2a2, drop_2b])
  ### Block_2c ###
  conv_2c = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2c = tf.keras.layers.BatchNormalization()(conv_2c)
  activation_2c = tf.keras.layers.Activation("relu")(bn_2c)
  mod_1 = module_1(activation_2c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2c2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2c2 = tf.keras.layers.BatchNormalization()(conv_2c2)
  drop_2c = tf.keras.layers.Dropout(0.2)(bn_2c2)
  add = tf.keras.layers.Add()([add, drop_2c])
  ### Block_2d ###
  conv_2d = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_2d = tf.keras.layers.BatchNormalization()(conv_2d)
  activation_2d = tf.keras.layers.Activation("relu")(bn_2d)
  mod_1 = module_1(activation_2d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_2d2 = tf.keras.layers.Conv3D(32, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_2d2 = tf.keras.layers.BatchNormalization()(conv_2d2)
  drop_2d = tf.keras.layers.Dropout(0.2)(bn_2d2)
  add = tf.keras.layers.Add()([add, drop_2d])


  ### Block_3a ###
  conv_3a = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
  activation_3a = tf.keras.layers.Activation("relu")(bn_3a)
  zero_3a = tf.keras.layers.ZeroPadding3D((0, 2, 2))(activation_3a)
  mod_1 = module_1(zero_3a, (1, 4, 4), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 192, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3a2 = tf.keras.layers.Conv3D(56, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
  ### Block_3b ###
  conv_3b = tf.keras.layers.Conv3D(336, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
  bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
  activation_3b = tf.keras.layers.Activation("relu")(bn_3b)
  mod_1 = module_1(activation_3b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 14, (1, 1, 1), (1, 1, 1), 336, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3b2 = tf.keras.layers.Conv3D(56, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
  drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
  add = tf.keras.layers.Add()([bn_3a2, drop_3b])
  ### Block_3c ###
  conv_3c = tf.keras.layers.Conv3D(336, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)  
  conv_3c = tf.keras.layers.Conv3D(336, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3c = tf.keras.layers.BatchNormalization()(conv_3c)
  activation_3c = tf.keras.layers.Activation("relu")(bn_3c)
  mod_1 = module_1(activation_3c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 14, (1, 1, 1), (1, 1, 1), 336, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3c2 = tf.keras.layers.Conv3D(56, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3c2 = tf.keras.layers.BatchNormalization()(conv_3c2)
  drop_3c = tf.keras.layers.Dropout(0.2)(bn_3c2)
  add = tf.keras.layers.Add()([add, drop_3c])
  ### Block_3d ###
  conv_3d = tf.keras.layers.Conv3D(336, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_3d = tf.keras.layers.BatchNormalization()(conv_3d)
  activation_3d = tf.keras.layers.Activation("relu")(bn_3d)
  mod_1 = module_1(activation_3d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 14, (1, 1, 1), (1, 1, 1), 336, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_3d2 = tf.keras.layers.Conv3D(56, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_3d2 = tf.keras.layers.BatchNormalization()(conv_3d2)
  drop_3d = tf.keras.layers.Dropout(0.2)(bn_3d2)
  add = tf.keras.layers.Add()([add, drop_3d])

  ### Block_4a ###
  conv_4a = tf.keras.layers.Conv3D(336, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
  activation_4a = tf.keras.layers.Activation("relu")(bn_4a)
  zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
  mod_1 = module_1(zero_4a, (1, 3, 3), (1, 2, 2), "relu")
  mod_2 = module_3(mod_1, 14, (1, 1, 1), (1, 1, 1), 336, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4a2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
  ### Block_4b ###
  conv_4b = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
  bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
  activation_4b = tf.keras.layers.Activation("relu")(bn_4b)
  mod_1 = module_1(activation_4b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4b2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
  drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
  add = tf.keras.layers.Add()([bn_4a2, drop_4b])
  ### Block_4c ###
  conv_4c = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
  activation_4c = tf.keras.layers.Activation("relu")(bn_4c)
  mod_1 = module_1(activation_4c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4c2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
  drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
  add = tf.keras.layers.Add()([add, drop_4c])
  ### Block_4d ###
  conv_4d = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4d = tf.keras.layers.BatchNormalization()(conv_4d)
  activation_4d = tf.keras.layers.Activation("relu")(bn_4d)
  mod_1 = module_1(activation_4d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4d2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4d2 = tf.keras.layers.BatchNormalization()(conv_4d2)
  drop_4d = tf.keras.layers.Dropout(0.2)(bn_4d2)
  add = tf.keras.layers.Add()([add, drop_4d])
  ### Block_4e ###
  conv_4e = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4e = tf.keras.layers.BatchNormalization()(conv_4e)
  activation_4e = tf.keras.layers.Activation("relu")(bn_4e)
  mod_1 = module_1(activation_4e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4e2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4e2 = tf.keras.layers.BatchNormalization()(conv_4e2)
  drop_4e = tf.keras.layers.Dropout(0.2)(bn_4e2)
  add = tf.keras.layers.Add()([add, drop_4e])
  ### Block_4f ###
  conv_4f = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_4f = tf.keras.layers.BatchNormalization()(conv_4f)
  activation_4f = tf.keras.layers.Activation("relu")(bn_4f)
  mod_1 = module_1(activation_4f, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_4f2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_4f2 = tf.keras.layers.BatchNormalization()(conv_4f2)
  drop_4f = tf.keras.layers.Dropout(0.2)(bn_4f2)
  add = tf.keras.layers.Add()([add, drop_4f])

  ### Block_5a ###
  conv_5a = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
  activation_5a = tf.keras.layers.Activation("relu")(bn_5a)
  mod_1 = module_1(activation_5a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5a2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
  ### Block_5b ###
  conv_5b = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
  bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
  activation_5b = tf.keras.layers.Activation("relu")(bn_5b)
  mod_1 = module_1(activation_5b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5b2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
  drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
  add = tf.keras.layers.Add()([bn_5a2, drop_5b])
  ### Block_5c ###
  conv_5c = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
  activation_5c = tf.keras.layers.Activation("relu")(bn_5c)
  mod_1 = module_1(activation_5c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5c2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
  drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
  add = tf.keras.layers.Add()([add, drop_5c])
  ### Block_5d ###
  conv_5d = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5d = tf.keras.layers.BatchNormalization()(conv_5d)
  activation_5d = tf.keras.layers.Activation("relu")(bn_5d)
  mod_1 = module_1(activation_5d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5d2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5d2 = tf.keras.layers.BatchNormalization()(conv_5d2)
  drop_5d = tf.keras.layers.Dropout(0.2)(bn_5d2)
  add = tf.keras.layers.Add()([add, drop_5d])
  ### Block_5e ###
  conv_5e = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5e = tf.keras.layers.BatchNormalization()(conv_5e)
  activation_5e = tf.keras.layers.Activation("relu")(bn_5e)
  mod_1 = module_1(activation_5e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5e2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5e2 = tf.keras.layers.BatchNormalization()(conv_5e2)
  drop_5e = tf.keras.layers.Dropout(0.2)(bn_5e2)
  add = tf.keras.layers.Add()([add, drop_5e])
  ### Block_5f ###
  conv_5f = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_5f = tf.keras.layers.BatchNormalization()(conv_5f)
  activation_5f = tf.keras.layers.Activation("relu")(bn_5f)
  mod_1 = module_1(activation_5f, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_5f2 = tf.keras.layers.Conv3D(160, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_5f2 = tf.keras.layers.BatchNormalization()(conv_5f2)
  drop_5f = tf.keras.layers.Dropout(0.2)(bn_5f2)
  add = tf.keras.layers.Add()([add, drop_5f])

  ### Block_6a ###
  conv_6a = tf.keras.layers.Conv3D(960, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
  activation_6a = tf.keras.layers.Activation("relu")(bn_6a)
  zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
  mod_1 = module_1(zero_6a, (1, 3, 3), (2, 2, 2), "relu")
  mod_2 = module_3(mod_1, 40, (1, 1, 1), (1, 1, 1), 960, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6a2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
  ### Block_6b ###
  conv_6b = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
  bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
  activation_6b = tf.keras.layers.Activation("relu")(bn_6b)
  mod_1 = module_1(activation_6b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6b2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
  drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
  add = tf.keras.layers.Add()([bn_6a2, drop_6b])
  ### Block_6c ###
  conv_6c = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
  activation_6c = tf.keras.layers.Activation("relu")(bn_6c)
  mod_1 = module_1(activation_6c, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6c2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
  drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
  add = tf.keras.layers.Add()([add, drop_6c])
  ### Block_6d ###
  conv_6d = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
  activation_6d = tf.keras.layers.Activation("relu")(bn_6d)
  mod_1 = module_1(activation_6d, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6d2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
  drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
  add = tf.keras.layers.Add()([add, drop_6d])
  ### Block_6e ###
  conv_6e = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6e = tf.keras.layers.BatchNormalization()(conv_6e)
  activation_6e = tf.keras.layers.Activation("relu")(bn_6e)
  mod_1 = module_1(activation_6e, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6e2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6e2 = tf.keras.layers.BatchNormalization()(conv_6e2)
  drop_6e = tf.keras.layers.Dropout(0.2)(bn_6e2)
  add = tf.keras.layers.Add()([add, drop_6e])
  ### Block_6f ###
  conv_6f = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6f = tf.keras.layers.BatchNormalization()(conv_6f)
  activation_6f = tf.keras.layers.Activation("relu")(bn_6f)
  mod_1 = module_1(activation_6f, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6f2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6f2 = tf.keras.layers.BatchNormalization()(conv_6f2)
  drop_6f = tf.keras.layers.Dropout(0.2)(bn_6f2)
  add = tf.keras.layers.Add()([add, drop_6f])
  ### Block_6g ###
  conv_6g = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6g = tf.keras.layers.BatchNormalization()(conv_6g)
  activation_6g = tf.keras.layers.Activation("relu")(bn_6g)
  mod_1 = module_1(activation_6g, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6g2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6g2 = tf.keras.layers.BatchNormalization()(conv_6g2)
  drop_6g = tf.keras.layers.Dropout(0.2)(bn_6g2)
  add = tf.keras.layers.Add()([add, drop_6g])
  ### Block_6h ###
  conv_6h = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_6h = tf.keras.layers.BatchNormalization()(conv_6h)
  activation_6h = tf.keras.layers.Activation("relu")(bn_6h)
  mod_1 = module_1(activation_6h, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_6h2 = tf.keras.layers.Conv3D(272, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_6h2 = tf.keras.layers.BatchNormalization()(conv_6h2)
  drop_6h = tf.keras.layers.Dropout(0.2)(bn_6h2)
  add = tf.keras.layers.Add()([add, drop_6h])

  ### Block_7a ###
  conv_7a = tf.keras.layers.Conv3D(1632, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
  bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
  activation_7a = tf.keras.layers.Activation("relu")(bn_7a)
  mod_1 = module_1(activation_7a, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 68, (1, 1, 1), (1, 1, 1), 1632, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7a2 = tf.keras.layers.Conv3D(448, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
  ### Block_7b ###
  conv_7b = tf.keras.layers.Conv3D(2668, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_7a2)
  bn_7b = tf.keras.layers.BatchNormalization()(conv_7b)
  activation_7b = tf.keras.layers.Activation("relu")(bn_7b)
  mod_1 = module_1(activation_7b, (1, 1, 1), (1, 1, 1), "relu")
  mod_2 = module_3(mod_1, 112, (1, 1, 1), (1, 1, 1), 2668, (1, 1, 1), (1, 1, 1))
  mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
  conv_7b2 = tf.keras.layers.Conv3D(448, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
  bn_7b2 = tf.keras.layers.BatchNormalization()(conv_7b2)
  drop_7b = tf.keras.layers.Dropout(0.2)(bn_7b2)
  add = tf.keras.layers.Add()([bn_7a2, drop_7b])

  ### END ###
  end = final(add, 1792, (1, 1, 1), (1, 1, 1), "valid", "relu")
  # gap = tf.keras.layers.GlobalAveragePooling
  return end

In [9]:
input_layer = tf.keras.layers.Input((64, 224, 224, 3))
anunay = EfficientNet4_3D(input_layer)
print(anunay.shape)

(None, 4, 8, 8, 1792)


In [10]:
model_4 = tf.keras.models.Model(inputs = input_layer, outputs = anunay)
model_4.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 224, 224 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 64, 224, 224, 0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 64, 224, 224, 7           rescaling_1[0][0]                
__________________________________________________________________________________________________
zero_padding3d_5 (ZeroPadding3D (None, 66, 226, 226, 0           normalization_1[0][0]            
_______________________________________________________________________________________

In [ ]:
import tensorflow as tf
import numpy as np
import random
import cv2
import os,sys
from PIL import Image
import scipy.misc


def capture_trim_resize(path, shape, n_frames, gray):
  out = []
  cap = cv2.VideoCapture(path)
  valid = True
  for i in range(n_frames):
    _, frame = cap.read()
    if not _:
      frame = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    frame = cv2.resize(frame, (shape[1], shape[0]))
    if gray:
      frame = frame[:, :, 0:2]
    out.append(frame)
  return np.array(out)


class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, x_data, y_data, frame_shape=None, n_frames=None, batch_size=None, shuffle=None):
    self.path = videos_folder_path
    self.n_frames = n_frames
    self.shape = frame_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.rgb_videos = []
    self.flow_videos = []
    self.labels = []

#     self.on_epoch_end()
    
#   def on_epoch_end(self):
#     if self.shuffle:
#       temp = [i for i in range(len(self.labels))]
#       random.shuffle(temp)
#       self.flow_videos = [self.flow_videos[i] for i in temp]
#       self.rgb_videos = [self.rgb_videos[i] for i in temp]
#       self.labels = [self.labels[i] for i in temp]
    

  def __getitem__(self, index):
    start_index = index * self.batch_size
    rgb = []
    flow = []
    y = []
    i = start_index - 1
    while len(y) < self.batch_size:
      try:
        i += 1
        video = capture_trim_resize(self.rgb_videos[i % len(self.rgb_videos)], self.shape, self.n_frames, False)
        op = capture_trim_resize(self.flow_videos[i % len(self.rgb_videos)], self.shape, self.n_frames, True)
        rgb.append(video.astype(np.float32) / 255)
        flow.append(op.astype(np.float32) / 255)
        y.append(self.labels[i % len(self.rgb_videos)])
      except Exception as err:
        print(err)
        continue
    return {"input_rgb":np.array(rgb), "input_flow":np.array(flow)}, np.array(y, dtype=np.float32) - 1

  def __len__(self):
    return int(np.floor(len(self.labels) / self.batch_size))

In [ ]:
train_data  = DataGenerator("/content/mydrive/MyDrive/hmdb51/hmdb51/dataset3", frame_shape=(224, 224), n_frames=64, batch_size=1, shuffle=True)

In [30]:
import os
import random
path = "/media/anunay/Elements/kinetics-dataset-downloader/train/train"
rgb_videos = []
flow_videos = []
labels = []
for i in os.listdir(path):
    for j in os.listdir(path + "/" + i):
#         rgb_videos.append(path + "/" + i + "/" + j)
        flow_videos.append(path + "/" + i + "/" + j)
        labels.append(i)

print(len(labels))
print(len(rgb_videos))
temp = [i for i in range(len(labels))]
random.shuffle(temp)
# flow_videos = [flow_videos[i] for i in temp]
rgb_videos = [rgb_videos[i] for i in temp]
# labels = [labels[i] for i in temp]

FileNotFoundError: [Errno 2] No such file or directory: '/media/anunay/Elements/kinetics-dataset-downloader/train/train'

In [26]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(['moving furniture', 'mowing lawn', 'news anchoring', 'opening bottle', 'opening present', 'paragliding', 'parasailing', 'parkour', 'passing American football (in game)', 'passing American football (not in game)', 'peeling apples', 'peeling potatoes', 'petting animal (not cat)', 'petting cat', 'picking fruit', 'planting trees', 'plastering', 'playing accordion', 'playing badminton', 'playing bagpipes', 'playing basketball', 'playing cards', 'playing cello', 'playing chess', 'playing clarinet', 'playing controller', 'playing cricket', 'playing cymbals', 'playing didgeridoo', 'playing drums', 'playing flute', 'playing guitar', 'playing harmonica', 'playing harp', 'playing ice hockey', 'playing keyboard', 'playing kickball', 'playing monopoly', 'playing paintball', 'playing piano', 'playing poker', 'playing recorder', 'playing saxophone', 'playing squash or racquetball', 'playing tennis', 'playing trombone', 'playing trumpet', 'playing ukulele', 'playing violin', 'playing volleyball', 'playing xylophone', 'pole vault', 'presenting weather forecast', 'pull ups', 'pumping fist', 'punching bag', 'punching person (boxing)', 'push up', 'pushing car', 'pushing cart', 'pushing wheelchair', 'reading book', 'reading newspaper', 'recording music', 'riding a bike', 'riding camel', 'riding elephant', 'riding mechanical bull', 'riding mountain bike', 'riding mule', 'riding or walking with horse', 'riding scooter', 'riding unicycle', 'ripping paper', 'robot dancing', 'rock climbing', 'rock scissors paper', 'roller skating', 'running on treadmill', 'sailing', 'salsa dancing', 'sanding floor', 'scrambling eggs', 'scuba diving', 'setting table', 'playing bass guitar', 'playing organ', 'pumping gas', 'shaking hands', 'skateboarding', 'shaking head', 'sharpening knives', 'sharpening pencil', 'shaving head', 'shaving legs', 'shearing sheep', 'shining shoes', 'shooting basketball', 'shooting goal (soccer)', 'shot put', 'shoveling snow', 'shredding paper', 'shuffling cards', 'side kick', 'sign language interpreting', 'singing', 'situp', 'ski jumping', 'skiing (not slalom or crosscountry)', 'skiing crosscountry', 'skiing slalom', 'skipping rope', 'skydiving', 'slacklining', 'slapping', 'sled dog racing', 'smoking', 'smoking hookah', 'snatch weight lifting', 'sneezing', 'sniffing', 'snorkeling', 'snowboarding', 'snowkiting', 'snowmobiling', 'somersaulting', 'spinning poi', 'spray painting', 'spraying', 'springboard diving', 'squat', 'sticking tongue out', 'stomping grapes', 'stretching arm', 'stretching leg', 'strumming guitar', 'surfing crowd', 'surfing water', 'sweeping floor', 'swimming backstroke'])

LabelBinarizer()

In [27]:
labels = lb.transform(labels)

In [29]:
print(labels.shape)

(54732, 140)


In [ ]:
import tenso